### Notebook 2 - trade graphs construction

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from networkx.readwrite import json_graph
import json
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
# aggregated trade dataset
df = pd.read_csv('data/preprocessed/trade_1988_2021.csv')

In [3]:
for y in tqdm(sorted(pd.unique(df['Year']))):
    y_df = df.loc[df['Year'] == y].reset_index(drop=True)
    
    # initialize directed graph
    graph = nx.DiGraph(year=int(y))
    
    # reporters or partners to exclude from the network
    to_exclude = ['European Union', 'Other Asia, nes', 'Other Asia', 'Reunion', 'Unspecified', 
                  ' World', 'World', 'Curaзao']
    
    # iteratively add nodes for each year
    for i in range(y_df.shape[0]):
        reporter = y_df['ReporterName'].iloc[i]
        partner = y_df['PartnerName'].iloc[i]
        if reporter not in to_exclude:
            if partner not in to_exclude:
                u = y_df['ReporterISO3'].iloc[i]
                v = y_df['PartnerISO3'].iloc[i]
                w = y_df['TradeValue in 1000 USD'].iloc[i]
                graph.add_edge(u, v)
                graph[u][v]['TradeValue in 1000 USD'] = w
    
    # save graph in json format
    with open(f'data/preprocessed/trade_graphs/export_graph_{int(y)}.json', 'w') as file:
        json.dump(json_graph.node_link_data(graph), file, indent=4)
        
    del graph

  0%|          | 0/34 [00:00<?, ?it/s]